# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D

# Train

In [2]:
train_set = pd.read_csv('train/train_encoded.csv')

In [3]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Model

In [4]:
batch_size = 1
num_classes = 2
epochs = 10

In [5]:
X_train

,id,len_keyword,len_location,len_text,len_text_original,diff_len_text,keyword_sum,keyword_min,keyword_max,keyword_mean,...,text_sum_tf-idf,text_min_tf-idf,text_max_tf-idf,text_mean_tf-idf,text_median_tf-idf,text_sum_w2v,text_min_w2v,text_max_w2v,text_mean_w2v,text_median_w2v
1792,2571,5.0,19.0,133,136,3,1.000000,1.000000,1.000000,1.000000,...,4.435130,0.095309,0.290488,0.211197,0.215512,18.576392,-0.110215,1.000000,0.136591,0.111687
6266,8953,5.0,16.0,56,59,3,1.000000,1.000000,1.000000,1.000000,...,2.873730,0.123184,0.490274,0.287373,0.256420,3.981104,-0.006107,0.526877,0.189576,0.190110
5535,7895,12.0,30.0,63,85,22,1.381139,0.538539,0.842601,0.690570,...,2.774225,0.237212,0.446876,0.346778,0.352323,3.106818,-0.040285,0.385922,0.110958,0.101616
4257,6049,9.0,0.0,99,100,1,1.414214,0.707107,0.707107,0.707107,...,4.001200,0.092441,0.375319,0.210589,0.202156,14.815921,-0.023579,0.638369,0.141104,0.099020
3792,5384,10.0,0.0,125,126,1,1.394610,0.579978,0.814632,0.697305,...,5.934999,0.065305,0.507047,0.197833,0.168021,79.558034,-0.076708,1.000000,0.244794,0.234924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,767,8.0,17.0,96,96,0,1.000000,1.000000,1.000000,1.000000,...,4.266455,0.090075,0.365628,0.237025,0.235383,13.535387,-0.064513,1.000000,0.148741,0.112126
1889,2715,5.0,6.0,19,19,0,1.000000,1.000000,1.000000,1.000000,...,1.700766,0.473721,0.720220,0.566922,0.506824,0.091488,0.091488,0.091488,0.091488,0.091488
3085,4427,10.0,0.0,86,109,23,1.000000,1.000000,1.000000,1.000000,...,3.907011,0.108332,0.401088,0.244188,0.242228,14.915197,-0.021130,1.000000,0.163903,0.125765
3603,5144,5.0,0.0,136,136,0,1.000000,1.000000,1.000000,1.000000,...,5.999714,0.060230,0.413988,0.249988,0.260803,37.602598,-0.060070,1.000000,0.179060,0.134188


In [6]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [7]:
X_train.shape

(5329, 26, 1)

In [8]:
input_shape = (X_train.shape[1], 1)
input_shape

(26, 1)

## Relu

In [9]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 25, 128)           384       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 25, 64)            8256      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 8, 64)             0         
_________________________________________________________________
dropout (Dropout)            (None, 8, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [11]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 17s 3ms/step - loss: 7.7248 - accuracy: 0.4828 - val_loss: 7.6559 - val_accuracy: 0.4308
Epoch 2/5
5329/5329 [==============================] - 17s 3ms/step - loss: 7.6160 - accuracy: 0.4789 - val_loss: 7.6585 - val_accuracy: 0.4304
Epoch 3/5
5329/5329 [==============================] - 17s 3ms/step - loss: 7.6518 - accuracy: 0.4806 - val_loss: 7.6609 - val_accuracy: 0.4304
Epoch 4/5
5329/5329 [==============================] - 17s 3ms/step - loss: 7.7043 - accuracy: 0.4830 - val_loss: 7.6609 - val_accuracy: 0.4304
Epoch 5/5
5329/5329 [==============================] - 19s 4ms/step - loss: 7.7943 - accuracy: 0.4804 - val_loss: 7.6583 - val_accuracy: 0.4304


In [12]:
model = Sequential()
model.add(Conv1D(128, kernel_size=3,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(64, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 24, 128)           512       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6, 128)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 128)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 6, 128)            16512     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 6, 32)             4128      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 3, 32)             0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 3, 64)            

In [14]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.3310 - accuracy: 0.5296 - val_loss: 6.6908 - val_accuracy: 0.5670
Epoch 2/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.1669 - accuracy: 0.5342 - val_loss: 6.6910 - val_accuracy: 0.5670
Epoch 3/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.1460 - accuracy: 0.5418 - val_loss: 6.6915 - val_accuracy: 0.5670
Epoch 4/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.3077 - accuracy: 0.5352 - val_loss: 6.6917 - val_accuracy: 0.5670
Epoch 5/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.1936 - accuracy: 0.5391 - val_loss: 6.6979 - val_accuracy: 0.5665


In [15]:
model = Sequential()
model.add(Conv1D(1024, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=10))
model.add(Dropout(0.25))
model.add(Dense(512))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(216, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(128, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(32, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 26, 1024)          2048      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 2, 1024)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2, 1024)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 2, 512)            524800    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 2, 32)             16416     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 2, 32)             0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 2, 216)           

In [17]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 52s 10ms/step - loss: 7.9077 - accuracy: 0.5592 - val_loss: 7.6481 - val_accuracy: 0.5652
Epoch 2/5
5329/5329 [==============================] - 47s 9ms/step - loss: 7.8414 - accuracy: 0.5652 - val_loss: 7.6482 - val_accuracy: 0.5652
Epoch 3/5
5329/5329 [==============================] - 51s 10ms/step - loss: 7.7421 - accuracy: 0.5667 - val_loss: 7.6460 - val_accuracy: 0.5652
Epoch 4/5
5329/5329 [==============================] - 50s 9ms/step - loss: 7.7585 - accuracy: 0.5648 - val_loss: 7.6454 - val_accuracy: 0.5652
Epoch 5/5
5329/5329 [==============================] - 47s 9ms/step - loss: 7.6948 - accuracy: 0.5740 - val_loss: 7.6457 - val_accuracy: 0.5652


## Softmax

In [18]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='softmax',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='softmax'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [19]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 19s 4ms/step - loss: 0.6933 - accuracy: 0.5637 - val_loss: 0.6932 - val_accuracy: 0.5652
Epoch 2/5
5329/5329 [==============================] - 19s 4ms/step - loss: 0.6932 - accuracy: 0.5444 - val_loss: 0.6932 - val_accuracy: 0.5652
Epoch 3/5
5329/5329 [==============================] - 18s 3ms/step - loss: 0.6932 - accuracy: 0.5331 - val_loss: 0.6931 - val_accuracy: 0.5652
Epoch 4/5
5329/5329 [==============================] - 19s 3ms/step - loss: 0.6932 - accuracy: 0.5215 - val_loss: 0.6931 - val_accuracy: 0.5652
Epoch 5/5
5329/5329 [==============================] - 20s 4ms/step - loss: 0.6932 - accuracy: 0.4995 - val_loss: 0.6931 - val_accuracy: 0.5652


## Tanh

In [20]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='tanh',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='tanh'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='tanh'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 17s 3ms/step - loss: 3.1980 - accuracy: 0.4680 - val_loss: 1.2163 - val_accuracy: 0.4343
Epoch 2/5
5329/5329 [==============================] - 17s 3ms/step - loss: 2.9906 - accuracy: 0.4896 - val_loss: 0.8546 - val_accuracy: 0.4221
Epoch 3/5
5329/5329 [==============================] - 18s 3ms/step - loss: 2.9181 - accuracy: 0.4916 - val_loss: 0.7774 - val_accuracy: 0.4067
Epoch 4/5
5329/5329 [==============================] - 18s 3ms/step - loss: 2.8016 - accuracy: 0.4894 - val_loss: 0.7587 - val_accuracy: 0.4111
Epoch 5/5
5329/5329 [==============================] - 18s 3ms/step - loss: 2.6992 - accuracy: 0.4802 - val_loss: 0.7366 - val_accuracy: 0.4181


## Text w2v

In [3]:
train_set = pd.read_csv('train/text_encoded_w2v.csv')

In [4]:
X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Model

In [5]:
batch_size = 1
num_classes = 2
epochs = 10

In [6]:
X_train

,id,text_sum,text_min,text_max,text_mean,text_median
5535,7895,1.885115,-0.031004,0.262091,0.089767,0.059510
6018,8596,9.313427,-0.040400,1.000000,0.102345,0.069088
4739,6738,14.726642,0.012219,1.000000,0.188803,0.132279
3181,4567,3.985746,-0.040585,0.433890,0.110715,0.082433
818,1189,3.576633,-0.090845,0.443895,0.170316,0.168023
...,...,...,...,...,...,...
5282,7548,2.514225,-0.025919,1.000000,0.119725,0.074594
563,814,4.079575,-0.013456,1.000000,0.271972,0.073396
2089,3003,12.973744,-0.108944,1.000000,0.142569,0.114495
3221,4622,3.870319,-0.074206,0.450665,0.107509,0.085664


In [7]:
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

In [8]:
X_train.shape

(5329, 6, 1)

In [9]:
input_shape = (X_train.shape[1], 1)
input_shape

(6, 1)

## Relu

In [10]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 5, 128)            384       
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5, 64)             8256      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 64)             0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [12]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 13s 2ms/step - loss: 8.2134 - accuracy: 0.4460 - val_loss: 8.4215 - val_accuracy: 0.4435
Epoch 2/5
5329/5329 [==============================] - 11s 2ms/step - loss: 8.2596 - accuracy: 0.4466 - val_loss: 8.4191 - val_accuracy: 0.4435
Epoch 3/5
5329/5329 [==============================] - 12s 2ms/step - loss: 8.1505 - accuracy: 0.4399 - val_loss: 8.4215 - val_accuracy: 0.4435
Epoch 4/5
5329/5329 [==============================] - 12s 2ms/step - loss: 8.2660 - accuracy: 0.4384 - val_loss: 8.4215 - val_accuracy: 0.4435
Epoch 5/5
5329/5329 [==============================] - 12s 2ms/step - loss: 8.2680 - accuracy: 0.4372 - val_loss: 8.4216 - val_accuracy: 0.4435


In [14]:
model = Sequential()
model.add(Conv1D(128, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.25))
model.add(Dense(128))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(64, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 6, 128)            256       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 6, 128)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 128)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 6, 128)            16512     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 6, 32)             4128      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 6, 32)             0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 6, 64)            

In [16]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.9467 - accuracy: 0.5384 - val_loss: 8.3027 - val_accuracy: 0.5565
Epoch 2/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.9275 - accuracy: 0.5485 - val_loss: 8.2378 - val_accuracy: 0.5565
Epoch 3/5
5329/5329 [==============================] - 14s 3ms/step - loss: 7.8626 - accuracy: 0.5372 - val_loss: 8.1313 - val_accuracy: 0.5565
Epoch 4/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.8740 - accuracy: 0.5393 - val_loss: 7.7117 - val_accuracy: 0.5565
Epoch 5/5
5329/5329 [==============================] - 13s 2ms/step - loss: 7.8631 - accuracy: 0.5378 - val_loss: 7.4220 - val_accuracy: 0.5565


In [17]:
model = Sequential()
model.add(Conv1D(1024, kernel_size=1,
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.25))
model.add(Dense(512))
model.add(Conv1D(filters=32, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(216, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(128, 1))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(64, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Conv1D(32, 1, activation='relu'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(num_classes, activation='relu'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 6, 1024)           2048      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 6, 1024)           0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 6, 1024)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 6, 512)            524800    
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 6, 32)             16416     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 6, 32)             0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 6, 216)           

In [19]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 47s 9ms/step - loss: 8.0898 - accuracy: 0.4265 - val_loss: 7.5757 - val_accuracy: 0.4426
Epoch 2/5
5329/5329 [==============================] - 51s 10ms/step - loss: 7.9531 - accuracy: 0.4260 - val_loss: 7.6082 - val_accuracy: 0.4426
Epoch 3/5
5329/5329 [==============================] - 50s 9ms/step - loss: 7.8164 - accuracy: 0.4218 - val_loss: 7.6192 - val_accuracy: 0.4426
Epoch 4/5
5329/5329 [==============================] - 47s 9ms/step - loss: 7.7759 - accuracy: 0.4256 - val_loss: 7.6203 - val_accuracy: 0.4431
Epoch 5/5
5329/5329 [==============================] - 48s 9ms/step - loss: 7.6847 - accuracy: 0.4237 - val_loss: 7.6232 - val_accuracy: 0.4431


## Softmax

In [20]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='softmax',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='softmax'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='softmax'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 13s 2ms/step - loss: 0.6932 - accuracy: 0.4915 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 2/5
5329/5329 [==============================] - 12s 2ms/step - loss: 0.6932 - accuracy: 0.4992 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 3/5
5329/5329 [==============================] - 11s 2ms/step - loss: 0.6932 - accuracy: 0.5050 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 4/5
5329/5329 [==============================] - 12s 2ms/step - loss: 0.6932 - accuracy: 0.4864 - val_loss: 0.6931 - val_accuracy: 0.4435
Epoch 5/5
5329/5329 [==============================] - 12s 2ms/step - loss: 0.6932 - accuracy: 0.5042 - val_loss: 0.6931 - val_accuracy: 0.4435


## Tanh

In [22]:
model = Sequential()
model.add(Conv1D(128, kernel_size=2,
                 activation='tanh',
                 input_shape=input_shape))
model.add(Conv1D(64, 1, activation='tanh'))
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='tanh'))

model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [23]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/5
5329/5329 [==============================] - 11s 2ms/step - loss: 3.2916 - accuracy: 0.4843 - val_loss: 0.8663 - val_accuracy: 0.4426
Epoch 2/5
5329/5329 [==============================] - 11s 2ms/step - loss: 3.1247 - accuracy: 0.4881 - val_loss: 0.8411 - val_accuracy: 0.4435
Epoch 3/5
5329/5329 [==============================] - 12s 2ms/step - loss: 3.1101 - accuracy: 0.4935 - val_loss: 0.8166 - val_accuracy: 0.4448
Epoch 4/5
5329/5329 [==============================] - 12s 2ms/step - loss: 3.0732 - accuracy: 0.4817 - val_loss: 0.8006 - val_accuracy: 0.4453
Epoch 5/5
5329/5329 [==============================] - 12s 2ms/step - loss: 2.9780 - accuracy: 0.5044 - val_loss: 0.7842 - val_accuracy: 0.4448
